# Sales Analyzer & Price Comparator


In [1]:

import pandas as pd

# Load the CSV files
woolies_df = pd.read_csv('customer_transactions_woolies.csv')
coles_df = pd.read_csv('customer_transactions_coles.csv')


## Keyword-Based Sales Analyzer

In [2]:

def keyword_sales_summary(keyword, woolies_df, coles_df):
    
    # Combine both store data into one DataFrame and label the source store
    all_data = pd.concat([woolies_df.assign(Store="Woolworths"), coles_df.assign(Store="Coles")])
    
    # Filter the combined data for items that contain the keyword in their name (case-insensitive)
    filtered = all_data[all_data["ItemName"].str.contains(keyword, case=False, na=False)]
    
# Group the filtered data by item name and summarize:
    # - Total quantity sold
    # - Number of unique customers who bought it
    # - Number of different stores it appeared in
    summary = (
        filtered.groupby("ItemName")
        .agg(
            TotalSold=("Quantity", "sum"),
            UniqueCustomers=("CustomerID", pd.Series.nunique),
            StoreCount=("Store", "nunique")
        )
        .sort_values("TotalSold", ascending=False) # Sort items by how much was sold, descending
        .reset_index()
    )
    
    # If the summary is not empty, mark the top-selling item
    if not summary.empty:
        summary["Remark"] = ""  # Create a new column to annotate
        summary.loc[0, "Remark"] = "Most Sold"   # Flag the top item as the most sold
    
    # Save the result to a CSV file
    summary.to_csv("keyword_sales_summary.csv", index=False)
    return summary


In [6]:
# Prompt the user to enter a keyword to search products by name
keyword = input("Enter a keyword (Product name) to analyze product sales: ").strip()

# Call the function to analyze products that match the keyword and get the summary DataFrame
summary_df = keyword_sales_summary(keyword, woolies_df, coles_df)

# Display the keyword and corresponding summary
print(f"Results for keyword: '{keyword}'")
print(summary_df)

# Suggestion to compare partially matched products across stores
# Extract all item names that matched the keyword search
matching_products = summary_df["ItemName"].tolist()

# Print each matching product to guide the user for further comparison or selection
print("\nProducts matching the keyword across stores:")
for product in matching_products:
    print("-", product)


Results for keyword: 'chips'
                                              ItemName  TotalSold  \
0              KB's Seafood Crumbed Squid Chips | 200g        375   
1    Old El Paso Nachips Tortilla Chips Cheese Flav...        105   
2                Orion Turtle Chips Choco Churros 160g         95   
3                  Mission White Strip Corn Chips 500g         89   
4              Calbee Potato Chips Pizza Flavoured 55g         84   
..                                                 ...        ...   
368           Coles Stone Ground Tortilla Chips | 400g          5   
369        Noshu 95% Sugar Free Choc Baking Chips 140g          5   
370  Vege Chips Deli Crisps Yellow Sweet Potato | 100g          5   
371         Wicked Sweet Heat Pickle Chips Mild | 500g          5   
372      Kettle Natural Potato Chips Fig & Brie | 150g          3   

     UniqueCustomers  StoreCount     Remark  
0                158           1  Most Sold  
1                 44           1             
2   

## Price Comparison for a Specific Product

In [4]:

def compare_prices(product_name, woolies_df, coles_df):
    # Normalize column names to avoid KeyErrors due to hidden whitespace or casing
    woolies_df.columns = woolies_df.columns.str.strip()
    coles_df.columns = coles_df.columns.str.strip()

    # Add Store column and combine both DataFrames
    woolies = woolies_df.assign(Store="Woolworths")
    coles = coles_df.assign(Store="Coles")
    combined = pd.concat([woolies, coles])

    # Check for 'ItemName' column
    if 'ItemName' not in combined.columns or 'UnitPrice' not in combined.columns:
        print("Required columns 'ItemName' or 'UnitPrice' are missing.")
        print(f"Available columns: {combined.columns.tolist()}")
        return

    # Perform case-insensitive partial matching
    matches = combined[combined["ItemName"].str.contains(product_name, case=False, na=False)]
    
    # Handle the case where no products matched the input keyword
    if matches.empty:
        print("No matching products found.")
        return

    print(f" Found {len(matches)} matching entries. Grouping by product and store...")

    # Group and sort by price
    grouped = (
        matches.groupby(["ItemName", "Store"])["UnitPrice"]
        .min()            # Get the minimum price per product/store combo
        .reset_index()    # Flatten the groupby object into a DataFrame
        .sort_values("UnitPrice")       # Sort from cheapest to most expensive
    )

    # Print cheapest option
    cheapest = grouped.iloc[0]   # Get the top row — the cheapest entry after sorting
    
    # Display the best match details clearly
    print(f"\n Cheapest match is:\n'{cheapest['ItemName']}' at {cheapest['Store']} for ${cheapest['UnitPrice']:.2f}")
    
    # Also display all the grouped results so the user can compare other options
    print("\n All matches:")
    print(grouped)


In [5]:
#Prompt the user to enter part of the product name for flexible price comparison
product_name = input("Enter the exact product name for price comparison: ").strip()
compare_prices(product_name, woolies_df, coles_df) #Run the price comparison using the updated compare_prices function


 Found 4825 matching entries. Grouping by product and store...

 Cheapest match is:
'Nong Shim Shrimp Meat Chip Shrimp Meat Chips 75g' at Woolworths for $1.45

 All matches:
                                              ItemName       Store  UnitPrice
150   Nong Shim Shrimp Meat Chip Shrimp Meat Chips 75g  Woolworths       1.45
293    Smiths Grainwaves Chips Sour Cream Chives | 40g       Coles       1.50
49                           Coles Banana Chips | 200g       Coles       2.00
175  Otep Brown Rice Chips With Pumpkin Caramelised...       Coles       2.00
190          Pringles Original Salted Potato Chips 53g  Woolworths       2.00
..                                                 ...         ...        ...
350        Vege Chips Natural Multipack 12 Pack | 250g       Coles       9.50
348                      Vege Chips Multi 12 Pack 250g  Woolworths       9.50
349                    Vege Chips Multi 12 Pack | 250g       Coles       9.50
93   Family Fave's Chicken Chips With Ciabatta